*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

googleapis-common-protos 1.56.2 requires protobuf<4.0.0dev,>=3.15.0, but you'll have protobuf 3.13.0 which is incompatible.
c:\users\husba\anaconda3\envs\tf_gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21500 sha256=120f039128ebf315ffa3638ebad029dc9498eab67535bb17ca21599a48f24d4b
  Stored in directory: c:\users\husba\appdata\local\pip\cache\wheels\29\93\c6\762e359f8cb6a5b69c72235d798804cae523bbe41c2aa8333d
Successfully built promise
2.3.0


In [3]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [4]:
train_data=pd.read_csv(train_file_path,sep="\t",header=None)
test_data=pd.read_csv(test_file_path,sep="\t",header=None)

train_data.columns = ['cc', 'text' ]
test_data.columns = ['cc', 'text' ]


train_data.cc = pd.Categorical(train_data.cc)
test_data.cc = pd.Categorical(test_data.cc)
train_data['code'] = train_data.cc.cat.codes
test_data['code'] = test_data.cc.cat.codes

train_data.tail()
test_data.tail()

,cc,text,code
1387,ham,true dear..i sat to pray evening and felt so.s...,0
1388,ham,"what will we do in the shower, baby?",0
1389,ham,where are you ? what are you doing ? are yuou ...,0
1390,spam,ur cash-balance is currently 500 pounds - to m...,1
1391,spam,not heard from u4 a while. call 4 rude chat pr...,1


In [5]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(train_data['text'])

X_train = tokenizer.texts_to_sequences(train_data['text'])
X_test = tokenizer.texts_to_sequences(test_data['text'])

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
y_train=train_data['code']
y_test=test_data['code']
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
print(X_train.shape,X_test.shape)

(4179, 100) (1392, 100)


In [10]:
embedding_dim = 32

model = keras.Sequential()
model.add(keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1 , 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           246176    
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                102432    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 348,641
Trainable params: 348,641
Non-trainable params: 0
_________________________________________________________________


In [11]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=10)

Epoch 1/10
418/418 [==============================] - 2s 4ms/step - loss: 0.1851 - accuracy: 0.9349 - val_loss: 0.0665 - val_accuracy: 0.9784
Epoch 2/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0435 - accuracy: 0.9885 - val_loss: 0.0576 - val_accuracy: 0.9806
Epoch 3/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0200 - accuracy: 0.9952 - val_loss: 0.0473 - val_accuracy: 0.9835
Epoch 4/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0117 - accuracy: 0.9971 - val_loss: 0.0483 - val_accuracy: 0.9856
Epoch 5/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.0506 - val_accuracy: 0.9864
Epoch 6/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0044 - accuracy: 0.9995 - val_loss: 0.0651 - val_accuracy: 0.9835
Epoch 7/10
418/418 [==============================] - 1s 3ms/step - loss: 0.0035 - accuracy: 0.9993 - val_loss: 0.0618 - val_accuracy: 0.9842
Epoch 

In [12]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):  
    pred_list=[pred_text]
    pred_data = tokenizer.texts_to_sequences(pred_list)
    pred_data = pad_sequences(pred_data, padding='post', maxlen=maxlen)
    pred_data=np.array(pred_data)
    # get the prediction
    predict= model.predict(pred_data)
    print(predict)
    if predict >= 0.5 :
        prediction=[predict,'spam']
    else:
        prediction=[predict,'ham']
    return (prediction)
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[[1.9469135e-06]]
[array([[1.9469135e-06]], dtype=float32), 'ham']


In [13]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[[1.9469135e-06]]
[[0.6815942]]
[[5.758235e-08]]
[[0.9994186]]
[[0.99959046]]
[[9.657623e-08]]
[[2.2246842e-07]]
You passed the challenge. Great job!
